# 说明

https://tianchi.aliyun.com/competition/entrance/231785/rankingList

https://zhuanlan.zhihu.com/p/127336206

## Background
本专题重点介绍曝光的公平性fariness of exposure，即如何推荐过去很少曝光的物品，以对抗推荐系统中经常遇到的马修效应。 尤其是，在对点击数据进行训练时，减少偏差对于该任务的成功至关重要。 就像现代推荐系统中记录的点击数据和实际在线环境之间存在差距一样，培训数据和测试数据之间也会存在差距，主要是关于趋势和项目的受欢迎程度。

* 获奖的解决方案需要在历史上很少接触的产品上表现良好。

* 培训数据和测试数据是在多个时期内收集的，甚至包括大规模的销售活动。 由于趋势的变化，对于可靠的预测不可避免地要执行偏差减小。

* 我们提供了商品的多模式功能以及一些（匿名的）关键用户功能，以帮助参与者探索能够抵抗数据偏差并能够很好地处理未开发商品的解决方案。

## Schedule
2020年3月30日：已发布培训数据样本集并开放注册。

2020年4月7日：发布了完整的培训和验证数据。

2020年4月13日：发布了（未标记）测试集A（仅用于开发，在确定最终的获奖解决方案时未考虑在内）。

2020年5月27日：报名截止日期。

2020年6月4日：（无标签）测试集B正式发布。

2020年6月11日：提交工作结束，提交技术报告的截止日期。

2020年6月20日：挑战赛获奖通知。

2020年8月23日至27日：KDD会议。

所有截止日期均为世界标准时间当天的晚上11:59。 如有必要，主办方保留更新比赛时间表的权利。

## Dataset
文件为CSV格式，采用UTF-8编码。 CSV文件的列可以是：

     item_id：商品的唯一标识符

     txt_vec：项目的文本特征，它是由预先训练的模型生成的128维实值向量

     img_vec：项目的图像特征，它是由预先训练的模型生成的128维实值向量

     user_id：用户的唯一标识符

     时间：点击事件发生的时间戳，即（（unix_timestamp-random_number_1）/ random_number_2

     user_age_level：用户所属的年龄段

     user_gender：用户的性别，可以为空

     user_city_level：用户所在城市的等级
     
     
数据收集时间超过十天，其中包括一次销售活动。 它涉及超过100万次点击，10万项和3万用户。 数据集的总大小约为500MB。

### 训练数据：underexpose_train.zip

underexpose_item_feat.csv的文件，其列为：item_id，txt_vec，img_vec

underexpose_user_feat.csv的文件，其列为：user_id，user_age_level，user_gender，user_city_level

它包含十个加密文件，其名称格式为underexpose_train_click-T.zip。 在这里T = 0,1,2，…，9表示我们处于比赛的阶段T。 比赛进入阶段T时，我们将在论坛中释放underexpose_train_click-T.zip的密码。underexpose_train_click-T.zip的内容为underexpose_train_click-T.csv，其列为：user_id，item_id，time

### 测试数据：underexpose_test.zip

它包含十个加密文件，其名称格式为underexpose_test_click-T.zip。 在这里T = 0,1,2，…，9表示我们处于比赛的阶段T。 当比赛进入阶段T时，我们将在论坛中释放underexpose_train_click-T.zip的密码。underexpose_test_click-T.zip的内容为underexpose_test_click-T.csv和underexpose_test_qtime-T.csv。

* underexpose_test_click-T.csv的列为：user_id，item_id，时间

* underexpose_test_qtime-T.csv的列为：user_id，query_time
这里的query_time是用户单击下一项的时间戳。 

这项比赛的任务是预测出现在underexpose_test_qtime-T.csv中的每个用户点击的下一项。 特别是，参与者需要为每个用户推荐五十个项目。 如果五十个推荐项目中的任何一个与实际情况相符，则参与者将获得positive score。

我们确保真实的下一项在underexpose_item_feat.csv中。 然而，在训练数据中可以观察到零次点击，尽管可能性不大。

## 提交
比赛进入阶段T时，参与者需要提交对underexpose_test_qtime-0,1,2，…，T.csv的预测。

     提交文件名：underexpose_submit-T.csv

     提交的文件应为51列的CSV文件。 不需要包括标题，即列的名称。 提交文件的51列应为：

         user_id，item_id_01，item_id_02，…，item_50

         这里item_id_01，item_id_02，…，item_id_50是为user_id推荐的五十个项目。 这五十个项目的顺序很重要。 请把最可能被用户点击的项目放在前面。 换句话说，item_id_01应该是最可能的。

         我们确保每个user_id不会出现在多个阶段中。 因此，您无需指定提交的每一行用于哪个阶段。

## Evaluation评价

以下官方提供的评估代码一目了然。
* user_id下一个要点击哪个item_id，必须要命中，才有分数。
* hitrate分数和rank(第几个命中)是没有关系的，
* NDCG则是rank越靠前，分数越高！
* hitrate和NDCG的关系:
* hitrage >= NDCG  只有所有的user_id预测item_id都第一个命中，等号才成立
* hitrage相同，NDCG较低，表示命中率相同，但是rank靠后




对于本次比赛，我们使用NDCG@50来衡量推荐列表的质量。

     我们将计算两个指标：NDCG@50-full和NDCG@50-rare。

         NDCG@50-full是对整个测试集（即underexpose_test_qtime-T.csv中的所有测试用例）进行计算的。

         在underexpose_test_qtime-T.csv中的一半测试用例上计算NDCG@50-rare。 所选的一半包括其下一个要预测的项目比过去训练集中的另一半更少探索的案例，即underexpose_train_click-0.zip，underexpose_train_click-1.zip，…，underexpose_train_click-T.zip。

     T = 0,1,2，…，6 用于开发。 参与者的最终排名将基于T = 7,8,9进行计算。

NDCG@50-full获胜团队需要跻身前10％，同时要在合格团队中获得最佳NDCG@50-rare。

### the official script for evalution, which is posted in the forum.
https://tianchi.aliyun.com/forum/postDetail?spm=5176.12586969.1002.3.6c3f5619NDeQ04&postId=102089
### [Must Read!] Official Script for Evaluation

In [7]:
# coding=utf-8
from __future__ import division
from __future__ import print_function

import datetime
import json
import sys
import time
from collections import defaultdict

import numpy as np

#### evaluate_each_phase

In [8]:
# the higher scores, the better performance
def evaluate_each_phase(predictions: dict, answers: dict):
    """
    
    predictions: dict
        key是user_id, value是list [pred_item_id_1,pred_item_id_2...]
    answers: dict
        key是user_id, value是(item_id, item_degree)
    """
    list_item_degress = []
    for user_id in answers:
        item_id, item_degree = answers[user_id]  # item_degree 应该是内置的表示新颖性的指标，越小，越新颖
        list_item_degress.append(item_degree)
    list_item_degress.sort()
    median_item_degree = list_item_degress[len(list_item_degress) // 2]

    num_cases_full = 0.0
    ndcg_50_full = 0.0
    ndcg_50_half = 0.0
    num_cases_half = 0.0
    hitrate_50_full = 0.0
    hitrate_50_half = 0.0
    for user_id in answers:
        item_id, item_degree = answers[user_id]
        rank = 0
        # 把正确的item_id和 预测值 依次 比对，
        while rank < 50 and predictions[user_id][rank] != item_id:
            rank += 1
        num_cases_full += 1.0
        if rank < 50:
            ndcg_50_full += 1.0 / np.log2(rank + 2.0)
            hitrate_50_full += 1.0
        if item_degree <= median_item_degree:
            num_cases_half += 1.0
            if rank < 50:
                ndcg_50_half += 1.0 / np.log2(rank + 2.0)
                hitrate_50_half += 1.0
    ndcg_50_full /= num_cases_full
    hitrate_50_full /= num_cases_full
    ndcg_50_half /= num_cases_half
    hitrate_50_half /= num_cases_half
    return np.array([ndcg_50_full, ndcg_50_half,
                     hitrate_50_full, hitrate_50_half], dtype=np.float32)

#### evaluate

In [9]:
# submit_fname is the path to the file submitted by the participants.
# debias_track_answer.csv is the standard answer, which is not released.
def evaluate(stdout, submit_fname,
             answer_fname='debias_track_answer.csv', current_time=None):
    schedule_in_unix_time = [
        0,  # ........ 1970-01-01 08:00:00 (T=0)
        1586534399,  # 2020-04-10 23:59:59 (T=1)
        1587139199,  # 2020-04-17 23:59:59 (T=2)
        1587743999,  # 2020-04-24 23:59:59 (T=3)
        1588348799,  # 2020-05-01 23:59:59 (T=4)
        1588953599,  # 2020-05-08 23:59:59 (T=5)
        1589558399,  # 2020-05-15 23:59:59 (T=6)
        1590163199,  # 2020-05-22 23:59:59 (T=7)
        1590767999,  # 2020-05-29 23:59:59 (T=8)
        1591372799  # .2020-06-05 23:59:59 (T=9)
    ]
    assert len(schedule_in_unix_time) == 10
    for i in range(1, len(schedule_in_unix_time) - 1):
        # 604800 == one week
        assert schedule_in_unix_time[i] + 604800 == schedule_in_unix_time[i + 1]

    if current_time is None:
        current_time = int(time.time())
    print('current_time:', current_time)
    print('date_time:', datetime.datetime.fromtimestamp(current_time))
    current_phase = 0
    while (current_phase < 9) and (
            current_time > schedule_in_unix_time[current_phase + 1]):
        current_phase += 1
    print('current_phase:', current_phase)

    try:
        answers = [{} for _ in range(10)]
        with open(answer_fname, 'r') as fin:
            for line in fin:
                line = [int(x) for x in line.split(',')]
                phase_id, user_id, item_id, item_degree = line
                assert user_id % 11 == phase_id
                # exactly one test case for each user_id
                answers[phase_id][user_id] = (item_id, item_degree)
    except Exception as _:
        return report_error(stdout, 'server-side error: answer file incorrect')

    try:
        predictions = {}
        with open(submit_fname, 'r') as fin:
            for line in fin:
                line = line.strip()
                if line == '':
                    continue
                line = line.split(',')
                user_id = int(line[0])
                if user_id in predictions:
                    return report_error(stdout, 'submitted duplicate user_ids')
                item_ids = [int(i) for i in line[1:]]
                if len(item_ids) != 50:
                    return report_error(stdout, 'each row need have 50 items')
                if len(set(item_ids)) != 50:
                    return report_error(
                        stdout, 'each row need have 50 DISTINCT items')
                predictions[user_id] = item_ids
    except Exception as _:
        return report_error(stdout, 'submission not in correct format')

    scores = np.zeros(4, dtype=np.float32)

    # The final winning teams will be decided based on phase T=7,8,9 only.
    # We thus fix the scores to 1.0 for phase 0,1,2,...,6 at the final stage.
    if current_phase >= 7:  # if at the final stage, i.e., T=7,8,9
        scores += 7.0  # then fix the scores to 1.0 for phase 0,1,2,...,6
    phase_beg = (7 if (current_phase >= 7) else 0)
    phase_end = current_phase + 1
    for phase_id in range(phase_beg, phase_end):
        for user_id in answers[phase_id]:
            if user_id not in predictions:
                return report_error(
                    stdout, 'user_id %d of phase %d not in submission' % (
                        user_id, phase_id))
        try:
            # We sum the scores from all the phases, instead of averaging them.
            scores += evaluate_each_phase(predictions, answers[phase_id])
        except Exception as _:
            return report_error(stdout, 'error occurred during evaluation')

    return report_score(
        stdout, score=float(scores[0]),
        ndcg_50_full=float(scores[0]), ndcg_50_half=float(scores[1]),
        hitrate_50_full=float(scores[2]), hitrate_50_half=float(scores[3]))

#### _create_answer_file_for_evaluation

In [12]:
# FYI. You can create a fake answer file for validation based on this. For example,
# you can mask the latest ONE click made by each user in underexpose_test_click-T.csv,
# and use those masked clicks to create your own validation set, i.e.,
# a fake underexpose_test_qtime_with_answer-T.csv for validation.
def _create_answer_file_for_evaluation(answer_fname='data_gen/debias_track_answer.csv'):
    train = './data_origin/underexpose_train/underexpose_train_click-%d.csv'
    test = './data_origin/underexpose_test/underexpose_test_click-%d/underexpose_test_click-%d.csv'

    # underexpose_test_qtime-T.csv contains only <user_id, item_id>
    # underexpose_test_qtime_with_answer-T.csv contains <user_id, item_id, time>
    answer = 'data_gen/underexpose_test_qtime_with_answer-%d.csv'  # not released

    item_deg = defaultdict(lambda: 0)
    now_phase = 2
    with open(answer_fname, 'w') as fout:
        for phase_id in range(now_phase+1):
            
            with open(train % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    item_deg[item_id] += 1
            with open(test % (phase_id, phase_id)) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    item_deg[item_id] += 1
            with open(answer % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    assert user_id % 11 == phase_id
                    print(phase_id, user_id, item_id, item_deg[item_id],
                          sep=',', file=fout)

In [13]:
_create_answer_file_for_evaluation()

# 思路

已知的数据包含 用户特征数据，item特征数据，user_item点击数据

任务：预测用户下一次点击哪个item?
## 思路1
baseline可以用UserCF和ItemCF来做，考虑到鱼佬说的，暂定用ItemCF来做
## 思路2
训练数据给出了很多的序列，可以按照NLG的思路做。

训练数据：
* abcdegf
* acdef
* ...

测试数据(?为待预测的数据):
* abc?
* cdf?

## 参考
### 鱼佬
https://zhuanlan.zhihu.com/p/127336206  
赛题主要考查如何消除人工智能偏见的问题，推荐历史点击次数少的商品。传统的召回方式，如协同过滤 item CF 和 user CF，user CF更加倾向于推荐热门商品，item CF推荐有很好的新颖性，很擅长推荐长尾里的物品，或许可以尝试一下。

根据向量相似性进行推荐，也是一个尝试的方向，不过看到用户前后点击的商品相似性并不高，让我有些迟疑。或许还需要更多的分析，如结合时间之类的属性。还有就是前后关系只能描述当前兴趣，可以尝试提取长期兴趣进行推荐。

还有就是深度学习模型进行召回，如YouTube的推荐系统算法，DSSM双塔模型等都是不错的方式。

对应排序阶段而言，一般都会上模型，需要考虑的就是特征，如何去区分热门商品，提高历史出现频次少的商品成为关键。要做到既能推荐对，又能推荐的够新颖。

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

import seaborn as sns
sns.set(font='Arial Unicode MS')  # 解决Seaborn中文显示问题
import sys
sys.path.append('/Users/luoyonggui/PycharmProjects/mayiutils_n1/mayiutils/data_prepare')
from data_explore import DataExplore as de

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# load data

In [3]:
path = './data_origin/'

## train_user_df

In [106]:
train_user_df = pd.read_csv(path+'underexpose_train/underexpose_user_feat.csv', names=['user_id','user_age_level','user_gender','user_city_level'])

train_user_df.head()

,user_id,user_age_level,user_gender,user_city_level
0,17,8.0,M,4.0
1,26,7.0,M,2.0
2,35,6.0,F,4.0
3,40,6.0,M,1.0
4,49,6.0,M,1.0


In [13]:
de.describe(train_user_df)

num of records: 6789, num of columns: 4


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
user_id,int64,6786,0,0.0,14818,2,0.0294594,17247.4,10062.6,10,8637,17196,25856,35432
user_age_level,float64,8,83,1.2,4,1425,20.9898,4.53579,1.80313,1,3,5,6,8
user_gender,object,2,81,1.2,F,5211,76.7565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_city_level,float64,6,22,0.3,6,1870,27.5446,3.70844,1.79852,1,2,3,6,6


In [105]:
import pandas as pd
import sys
sys.path.append('/Users/luoyonggui/PycharmProjects/mayiutils_n1/mayiutils/data_prepare')
from cat_feature_encoder import CatFeatureEncoder as cfe

In [110]:
train_user_df = cfe.build_one_hot_features(train_user_df, ['user_age_level', 'user_gender', 'user_city_level'])

In [121]:
train_user_df.head()

,user_id,user_age_level_1.0,user_age_level_2.0,user_age_level_3.0,user_age_level_4.0,user_age_level_5.0,user_age_level_6.0,user_age_level_7.0,user_age_level_8.0,user_gender_F,user_gender_M,user_city_level_1.0,user_city_level_2.0,user_city_level_3.0,user_city_level_4.0,user_city_level_5.0,user_city_level_6.0
0,17,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0
1,26,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0
2,35,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0
3,40,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
4,49,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0


In [119]:
train_user_df.to_pickle('data_gen/user_features.pkl')

## train_item_df

In [56]:
train_item_df = pd.read_csv(path+'underexpose_train/underexpose_item_feat.csv', sep=r',\s+|,\[|\],\[',names=['item_id']+list(range(256)))
train_item_df.iloc[:, -1] = train_item_df.iloc[:, -1].str.replace(']', '').map(float)

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [112]:
train_item_df.head()

,item_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,42844,4.514945,-2.383720,0.500414,0.407068,-1.995229,0.109078,-0.691775,2.227460,-6.437974,-0.824897,-0.138724,-0.379329,0.627660,0.418377,4.441218,0.299819,0.578557,-4.699289,-0.394740,-2.391651,0.370532,-1.355466,-1.074178,-2.321640,-0.332456,0.123886,-2.439156,-0.345599,-3.304347,1.485284,0.909802,-1.643002,5.037034,2.780115,4.776496,2.255275,3.769707,-3.661684,-0.649405,4.199636,-0.634806,2.430340,-2.874019,-0.786178,-0.504916,-6.007789,1.498495,1.530613,2.379655,...,0.312405,3.444607,-0.886250,-1.343637,0.954459,0.630835,-2.394722,0.683487,1.149004,-1.351173,2.023900,1.599198,1.382868,1.605678,1.880667,-0.508161,0.242840,-0.260849,1.875943,0.206135,0.186973,2.047446,-0.575472,3.016410,2.757146,3.353721,-0.457272,-0.125337,2.332963,3.858967,-2.075490,-0.705496,0.203452,1.719733,2.925039,-0.388639,1.225732,-1.773137,0.052655,1.279922,-3.374727,-1.506969,-1.820180,-3.024644,0.445263,0.013933,-1.300239,2.759948,2.056171,0.508703
1,67898,-2.002905,-0.929881,0.790017,-1.380895,-0.510463,-1.810096,1.363962,0.497401,-4.038903,-3.057872,0.758558,-1.012155,2.816802,2.086895,-1.464331,-1.840496,-2.089971,-1.566872,1.545390,1.284341,-2.270262,0.780126,1.615594,-0.546058,1.370750,-1.178124,1.346842,0.442434,-1.498540,-0.589944,2.008351,-0.497135,-1.644230,3.140623,3.492178,0.335395,1.810923,-4.012080,2.419593,0.190941,-0.630611,3.289332,-1.446719,-0.611340,0.700662,-2.465656,-0.596773,2.498210,3.682916,...,2.399420,2.024863,0.170483,-0.039203,-1.506677,-1.945932,-0.020228,-0.495499,-0.141013,-1.617521,2.624676,-2.581922,0.220891,0.328793,0.647758,0.231990,1.101486,1.079527,2.953102,-0.528682,-1.140600,-0.373299,0.109811,2.813541,0.596998,1.754836,-1.359771,0.466501,2.377417,-0.180653,-3.259304,0.120833,2.225643,2.220507,-1.178944,-0.821367,0.717239,-1.455829,-1.260584,2.623467,-0.538330,-2.620164,1.277195,0.601015,-0.345312,0.993457,1.351633,2.162675,2.768597,-0.937197
2,66446,4.221673,-1.497139,1.133570,-2.745607,-4.197045,-0.542392,-1.396256,1.838419,-6.066454,-2.191799,0.752804,0.868623,6.187662,1.725745,2.887859,-1.486026,-0.182256,-3.710785,1.512866,-0.636434,0.288435,-3.369717,-0.265998,-3.549319,3.375338,-0.901461,-1.558371,1.695343,-4.450464,0.545495,1.000096,-3.468751,3.327641,1.556890,4.493203,0.369089,0.167196,-4.837062,1.216016,4.699153,-1.094529,3.015942,-1.322741,-0.829172,0.555047,-5.592765,1.254898,3.182450,3.053574,...,-0.005492,3.827181,-0.358198,-2.009379,-0.224391,0.803851,-0.909498,0.962810,2.601583,0.056328,1.859474,-0.316134,-1.131286,1.701278,2.305405,-1.941271,1.248002,0.291000,0.792067,1.361166,1.129005,1.947404,-0.859423,2.023223,2.348651,4.506127,0.684437,2.064992,0.022901,3.464243,-2.325273,0.131324,-1.876178,1.770354,2.925176,-1.851054,-0.092587,-0.580742,-0.422019,0.923714,-4.582711,-1.056910,-2.568084,-2.038061,2.508719,-0.764789,-0.657116,3.252782,2.687366,0.844332
3,63651,2.657970,-0.941863,1.121529,-5.109496,-0.279041,-0.351968,-1.086983,2.703607,-6.494977,-0.746769,-0.068571,-3.894670,4.937046,-1.863204,-1.955068,1.900193,1.743841,-6.024790,1.460414,-2.206104,-1.997572,-3.414536,-0.178739,0.987313,1.255347,-1.187136,2.070518,2.191021,-2.936702,2.617733,0.919181,-3.087907,-0.358938,-0.428679,3.815598,2.440558,1.281061,-0.732530,1.517067,2.790302,-2.019122,2.419042,-2.044806,0.649187,1.940526,-4.965359,0.930460,-1.152011,0.167594,...,-0.733038,-0.913736,-1.296190,4.821739,0.687235,-1.406431,0.669184,-1.847598,-0.817075,-1.181172,2.588552,-1.123118,0.232427,2.170325,0.579414,2.601421,-0.596196,-1.798693,0.312326,0.387486,-2.207365,-1.029329,-1.274233,2.047230,1.928517,2.102633,-0.559383,-0.951418,-2.021749,1.366272,-1.947211,-2.114419,1.140394,-0.796024,1.9

In [117]:
train_item_df.head()

,item_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,42844,1.757887,-1.102202,-0.222023,1.575777,-0.736294,-0.168443,-0.051297,1.292102,-1.863637,0.122845,0.121319,0.516905,-1.193824,0.189392,2.478445,0.412654,0.590463,-1.778120,-0.182573,-0.525687,0.844209,0.017419,-1.115269,-0.708528,0.106707,0.490688,-2.188021,-0.081683,-1.391624,0.291258,0.958997,0.045490,2.991187,1.561606,1.793469,1.150937,1.878136,-0.970536,-0.984191,1.551317,0.474756,0.914999,-1.354062,-1.495628,-0.643347,-1.667325,0.323319,0.531047,0.737569,...,-0.195995,1.444276,-0.072901,-0.649905,0.313999,0.988210,-0.742056,0.822419,0.413943,-0.523966,0.065430,1.101006,0.419622,0.656972,0.506902,-0.520453,-0.740179,0.218208,0.464606,-0.310160,-0.139749,0.813226,-0.256569,0.956343,0.706974,1.194861,-0.077909,-0.663368,0.870748,1.940528,0.026493,0.518132,-0.527107,0.472771,0.963207,0.769440,0.503484,-0.332108,0.052038,0.863489,-1.786997,-0.056063,-1.056496,-0.952991,0.031248,-0.152827,-0.978874,0.416639,0.195811,0.423983
1,67898,-1.991244,-0.326516,-0.044938,0.515855,0.126385,-1.246482,0.941128,0.369639,-0.077474,-1.062310,0.632865,0.164666,-0.173332,1.115121,-0.520361,-0.847728,-1.042623,-0.056352,0.944852,1.635520,-0.599170,1.156619,0.436259,0.267531,1.075779,-0.211385,-0.000299,0.345218,-0.341474,-1.053010,1.513930,0.653496,-0.469300,1.745750,0.987036,0.233495,1.047592,-1.159734,0.511048,-0.522723,0.477134,1.437587,-0.652003,-1.404134,-0.084294,0.686014,-0.904300,1.039448,1.496823,...,1.124889,0.736584,0.545674,-0.009513,-1.045850,-0.668804,0.423577,0.171897,-0.275322,-0.681048,0.548542,-1.232298,-0.212655,0.004834,-0.140794,-0.092054,-0.245032,1.046985,1.073848,-0.769670,-0.949525,-0.597863,0.085385,0.776866,-0.633360,0.389113,-0.598649,-0.323668,0.895281,-0.135952,-1.063844,1.065588,0.598996,0.751578,-1.139146,0.487528,0.195173,-0.150884,-0.689160,1.808602,-0.216371,-0.787458,0.607339,1.105438,-0.344081,0.401650,0.457390,-0.167607,0.804866,-0.355152
2,66446,1.589194,-0.629173,0.165136,-0.293160,-2.015595,-0.534387,-0.391392,1.084667,-1.587032,-0.602641,0.629585,1.211532,1.398032,0.914748,1.689658,-0.638989,0.124860,-1.234778,0.925952,0.506249,0.799338,-1.057054,-0.649090,-1.383398,2.216329,-0.062202,-1.679064,1.023956,-2.058138,-0.317508,1.004609,-0.923267,2.105826,0.936795,1.615587,0.249596,0.350635,-1.605186,-0.075343,1.809761,0.214070,1.271264,-0.591021,-1.518127,-0.151818,-1.391590,0.180595,1.398965,1.130181,...,-0.397193,1.634975,0.236203,-0.976741,-0.337350,1.099469,-0.012963,0.976539,1.190066,0.306125,-0.066792,0.032141,-0.948426,0.705798,0.730033,-1.349935,-0.160542,0.559426,-0.148436,0.412128,0.434860,0.754910,-0.398259,0.077673,0.453511,1.775609,0.580855,0.593825,-0.404105,1.737629,-0.203567,1.072538,-1.685195,0.500954,0.963277,-0.183290,-0.295843,0.348906,-0.215870,0.612916,-2.455906,0.239636,-1.458252,-0.392869,1.010882,-0.593637,-0.630557,0.898723,0.735422,0.604839
3,63651,0.689737,-0.332909,0.157773,-1.694496,0.260846,-0.427423,-0.242087,1.545983,-1.906077,0.164312,0.161314,-1.439781,0.815043,-1.076477,-0.769554,1.355076,1.303593,-2.506697,0.895472,-0.416600,-0.450126,-1.080963,-0.598757,1.110443,1.010119,-0.216244,0.417873,1.292479,-1.177823,1.024824,0.963735,-0.721188,0.196403,-0.077413,1.190113,1.239477,0.822925,0.611071,0.071331,0.822149,-0.310218,0.908125,-0.946190,-0.744488,0.490659,-0.974751,-0.009493,-0.878472,-0.551132,...,-0.857661,-0.728202,-0.312866,2.376894,0.166350,-0.321873,0.762008,-0.574143,-0.636547,-0.423706,0.519493,-0.418202,-0.206378,0.945352,-0.176698,1.279369,-1.224019,-0.732667,-0.419778,-0.196754,-1.600217,-0.980272,-0.605248,0.098911,0.192824,0.564383,-0.136827,-1.137518,-1.532485,0.659212,0.144643,-0.415301,-0.005349,-0.927885,0

In [27]:
de.describe(train_item_df)

num of records: 108916, num of columns: 257


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
item_id,int64,108916,0,0.0,1.000000,1,0.000918139,58485.345982,33065.656602,1.000000,30189.750000,58643.500000,87014.250000,117538.000000
0,float64,108745,0,0.0,0.619635,3,0.00275442,1.458865,1.738504,-8.101768,0.285449,1.489091,2.647692,8.362688
1,float64,108810,0,0.0,-1.928095,3,0.00275442,-0.317904,1.874272,-8.306049,-1.594721,-0.405978,0.838072,8.153514
2,float64,108771,0,0.0,-1.808081,3,0.00275442,0.863508,1.635397,-7.136532,-0.162674,0.919081,1.920463,8.325392
3,float64,108649,0,0.0,-2.129149,3,0.00275442,-2.251081,1.686890,-9.212244,-3.422497,-2.393868,-1.209849,11.561003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,float64,108802,0,0.0,-0.310374,3,0.00275442,0.283913,1.766579,-10.643577,-0.726043,0.431549,1.425704,7.465464
252,float64,108791,0,0.0,-0.136049,3,0.00275442,0.507123,1.846375,-10.683113,-0.831946,0.641637,1.909708,6.809214
253,float64,108513,0,0.0,2.266967,3,0.00275442,2.334019,1.022302,-3.334639,1.690839,2.370883,3.020225,6.749975
254,float64,108610,0,0.0,1.178804,3,0.00275442,1.827126,1.169730,-5.783994,1.132481,1.855712,2.559368,7.211661


In [113]:
from sklearn.preprocessing import StandardScaler

In [115]:
scaler = StandardScaler().fit(train_item_df.iloc[:, 1:])

train_item_df.iloc[:, 1:] = scaler.transform(train_item_df.iloc[:, 1:])

In [118]:
de.describe(train_item_df)

num of records: 108916, num of columns: 257


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
item_id,int64,108916,0,0.0,1.000000,1,0.000918139,5.848535e+04,33065.656602,1.000000,30189.750000,58643.500000,87014.250000,117538.000000
0,float64,108745,0,0.0,-0.482733,3,0.00275442,1.666925e-17,1.000005,-5.499369,-0.674960,0.017387,0.683825,3.971146
1,float64,108810,0,0.0,-0.859107,3,0.00275442,5.616684e-18,1.000005,-4.262020,-0.681237,-0.046991,0.616763,4.519866
2,float64,108771,0,0.0,-1.633611,3,0.00275442,2.783203e-17,1.000005,-4.891827,-0.627485,0.033982,0.646302,4.562757
3,float64,108649,0,0.0,0.072283,3,0.00275442,-5.273651e-17,1.000005,-4.126645,-0.694427,-0.084645,0.617253,8.187938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,float64,108802,0,0.0,-0.336408,3,0.00275442,3.092062e-17,1.000005,-6.185708,-0.571704,0.083572,0.646332,4.065249
252,float64,108791,0,0.0,-0.348344,3,0.00275442,2.916430e-17,1.000005,-6.060678,-0.725245,0.072853,0.759646,3.413239
253,float64,108513,0,0.0,-0.065590,3,0.00275442,1.383131e-16,1.000005,-5.545017,-0.629152,0.036060,0.671239,4.319638
254,float64,108610,0,0.0,-0.554252,3,0.00275442,-4.116090e-17,1.000005,-6.506763,-0.593853,0.024438,0.625996,4.603251


In [120]:
train_item_df.to_pickle('data_gen/item_features.pkl')

# click_df

In [1]:
from myutils import *

In [2]:
now_phase = 2
whole_click, click_train, click_test, test_qtime = load_click_data(now_phase, gen_val_set=True)

phase: 0
gen val set...
phase: 1
gen val set...
phase: 2
gen val set...


In [14]:
whole_click.to_pickle('data_gen/whole_click_no_val_set.pkl')

In [15]:
whole_click.to_pickle('data_gen/test_qtime.pkl')

In [3]:
whole_click.shape

(457791, 3)

In [97]:
whole_click.shape

(459740, 3)

## 验证是否存在一个user重复点击多次相同item_id的情况

### whole_click

In [99]:
def t(s):
    return s.nunique()

In [100]:
whole_click['t'] = range(whole_click.shape[0])

In [101]:
dft = whole_click.groupby('user_id').agg(t).sort_values('item_id')

In [102]:
# 可以看到重复点击的情况很少！
dft[dft.item_id != dft.t]

,item_id,time,t
user_id,,,
13210,3,5.0,5
26768,3,4.0,4
21088,3,4.0,4
9318,4,5.0,5
15534,4,5.0,5
...,...,...,...
26540,163,178.0,178
4262,175,176.0,176
2072,188,197.0,198


In [131]:
del click_test['t']

In [132]:
click_test.sort_values('time').drop_duplicates('user_id', keep='last')

,user_id,item_id,time
541,22946,102303,0.983740
14900,22352,41533,0.983740
20690,20922,13603,0.983741
6518,34496,20710,0.983742
8907,15290,111807,0.983744
...,...,...,...
2127,4215,3014,0.984067
19616,8340,48809,0.984067
3583,22211,11830,0.984067
13272,7801,44841,0.984067


### click_test

In [103]:
click_test['t'] = range(click_test.shape[0])
dft = click_test.groupby('user_id').agg(t).sort_values('item_id')

In [104]:
# 在test中没有重复点击的情况
dft[dft.item_id != dft.t]

,item_id,time,t
user_id,,,


## test_click_df.user_id也在train_click_df中出现过

In [39]:
len(set(test_click_df.user_id).intersection(set(train_click_df.user_id)))

2866

## test_click_0_df.user_id有在train_click_1_df出现过，也就是可以phase1阶段的train_click数据来改进phase0阶段的结果

In [42]:
len(set(test_click_0_df.user_id).intersection(set(train_click_1_df.user_id)))

1402

In [12]:
test_click_0_df.head()

,user_id,item_id,time
0,1133,221,0.983812
1,17864,253,0.983783
2,6941,309,0.983785
3,34089,358,0.983781
4,21659,536,0.983793


In [11]:
de.describe(test_click_0_df)

num of records: 21216, num of columns: 3


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
user_id,int64,1663,0,0.0,35123.000000,98,0.461916,16435.225160,10072.244892,11.00000,7722.000000,15939.000000,24585.0000,35398.000000
item_id,int64,15670,0,0.0,113569.000000,19,0.0895551,56308.638763,34119.203232,1.00000,25690.500000,55536.500000,86055.7500,117069.000000
time,float64,20419,0,0.0,0.983876,4,0.0188537,0.983854,0.000061,0.98374,0.983794,0.983848,0.9839,0.983958


In [21]:
de.describe(test_click_1_df)

num of records: 24465, num of columns: 3


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
user_id,int64,1726,0,0.0,4643.000000,93,0.380135,16739.294666,9831.424398,1.000000,8449.000000,16248.000000,24982.000000,35421.000000
item_id,int64,17295,0,0.0,52766.000000,30,0.122624,56504.339792,34130.032894,23.000000,26155.000000,55472.000000,86155.000000,117283.000000
time,float64,23408,0,0.0,0.983818,3,0.0122624,0.983909,0.000061,0.983794,0.983849,0.983902,0.983955,0.984012


In [25]:
de.describe(test_click_df)

num of records: 45681, num of columns: 3


,Data Type,Unique Values,count Missing,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
user_id,int64,3389,0,0.0,35123.000000,98,0.214531,16598.073181,9945.043089,1.00000,8107.000000,16061.000000,24839.000000,35421.000000
item_id,int64,27195,0,0.0,52766.000000,47,0.102887,56413.448764,34124.769689,1.00000,25955.000000,55495.000000,86099.000000,117283.000000
time,float64,42497,0,0.0,0.983876,4,0.00875638,0.983883,0.000067,0.98374,0.983832,0.983885,0.983938,0.984012


In [29]:
test_click_df.groupby('user_id')['item_id'].count().describe()

count    3389.000000
mean       13.479197
std        12.119118
min         2.000000
25%         6.000000
50%        10.000000
75%        17.000000
max        98.000000
Name: item_id, dtype: float64

## 验证test_qtime_df的user_id对应的query_time，是否在click_df的user_id对应的time之后
惊奇的发现，不是最后，有的user_id对应的query_time < click_df的user_id对应的time

经过验证，各自对应的阶段是没有这种情况的。所以直接使用phaseT阶段train的数据来预测phaseT-1好像有点数据泄露的问题。__至少应该把T阶段train_click中的属于T阶段之前的userid大于query_time的数据删掉，再来用它预测T阶段之前的数据！__

In [32]:
for uid in test_qtime_df['user_id'].tolist():
    if click_df.loc[click_df['user_id']==uid].sort_values('time')['time'].iloc[-1] >= test_qtime_df[test_qtime_df['user_id']==uid]['query_time'].iloc[-1]:
        print(click_df.loc[click_df['user_id']==uid].sort_values('time').iloc[-1])
        print(test_qtime_df[test_qtime_df['user_id']==uid])

user_id       22.00000
item_id    63202.00000
time           0.98399
Name: 355952, dtype: float64
   user_id  query_time
1       22    0.983956
user_id       44.000000
item_id    42253.000000
time           0.984003
Name: 251960, dtype: float64
   user_id  query_time
2       44    0.983924
user_id      55.000000
item_id    3476.000000
time          0.984008
Name: 359451, dtype: float64
   user_id  query_time
3       55    0.983953
user_id        66.000000
item_id    115018.000000
time            0.983986
Name: 424959, dtype: float64
   user_id  query_time
4       66    0.983895
user_id       88.000000
item_id    93044.000000
time           0.984011
Name: 365031, dtype: float64
   user_id  query_time
5       88    0.983901
user_id      132.000000
item_id    88637.000000
time           0.984007
Name: 328231, dtype: float64
   user_id  query_time
7      132    0.983954
user_id      143.000000
item_id    28434.000000
time           0.984004
Name: 280850, dtype: float64
   user_id  query_ti

user_id     1881.00000
item_id    29035.00000
time           0.98401
Name: 375825, dtype: float64
    user_id  query_time
95     1881    0.983801
user_id     2035.000000
item_id    74938.000000
time           0.984006
Name: 246473, dtype: float64
     user_id  query_time
103     2035    0.983956
user_id      2057.000000
item_id    108666.000000
time            0.984001
Name: 314685, dtype: float64
     user_id  query_time
104     2057    0.983953
user_id     2079.000000
item_id    73319.000000
time           0.983996
Name: 385821, dtype: float64
     user_id  query_time
105     2079    0.983908
user_id     2123.000000
item_id    49524.000000
time           0.984007
Name: 398994, dtype: float64
     user_id  query_time
106     2123    0.983909
user_id      2134.000000
item_id    105153.000000
time            0.984005
Name: 248314, dtype: float64
     user_id  query_time
107     2134    0.983952
user_id     2145.000000
item_id    38501.000000
time           0.984009
Name: 390949, dtype: 

user_id     3927.000000
item_id    22424.000000
time           0.984012
Name: 389963, dtype: float64
     user_id  query_time
196     3927      0.9839
user_id      3949.000000
item_id    112334.000000
time            0.984005
Name: 263753, dtype: float64
     user_id  query_time
198     3949    0.983957
user_id     3960.000
item_id    20435.000
time           0.984
Name: 338605, dtype: float64
     user_id  query_time
199     3960    0.983926
user_id     3971.000000
item_id    75225.000000
time           0.984012
Name: 246494, dtype: float64
     user_id  query_time
200     3971    0.983872
user_id      3993.000
item_id    109069.000
time            0.984
Name: 454139, dtype: float64
     user_id  query_time
201     3993    0.983954
user_id      4004.000000
item_id    102584.000000
time            0.983998
Name: 416978, dtype: float64
     user_id  query_time
202     4004    0.983955
user_id     4015.000000
item_id    89335.000000
time           0.983983
Name: 452949, dtype: float64
  

user_id     5632.000000
item_id    74228.000000
time           0.984002
Name: 261359, dtype: float64
     user_id  query_time
297     5632    0.983951
user_id    5643.000000
item_id    3460.000000
time          0.984006
Name: 344669, dtype: float64
     user_id  query_time
298     5643    0.983955
user_id     5665.000000
item_id    21557.000000
time           0.984009
Name: 265553, dtype: float64
     user_id  query_time
299     5665    0.983954
user_id     5676.000000
item_id    67285.000000
time           0.984007
Name: 283162, dtype: float64
     user_id  query_time
300     5676      0.9839
user_id     5731.000000
item_id    44290.000000
time           0.983987
Name: 376747, dtype: float64
     user_id  query_time
303     5731    0.983934
user_id     5764.000000
item_id    53686.000000
time           0.983919
Name: 326203, dtype: float64
     user_id  query_time
305     5764    0.983918
user_id    5841.000000
item_id    9537.000000
time          0.984011
Name: 301334, dtype: float64

user_id     7513.00000
item_id    14403.00000
time           0.98398
Name: 287248, dtype: float64
     user_id  query_time
393     7513     0.98394
user_id     7535.00000
item_id    63736.00000
time           0.98401
Name: 377888, dtype: float64
     user_id  query_time
394     7535     0.98395
user_id     7546.000000
item_id    36152.000000
time           0.983961
Name: 361619, dtype: float64
     user_id  query_time
395     7546    0.983929
user_id    7612.000000
item_id    5013.000000
time          0.983988
Name: 308332, dtype: float64
     user_id  query_time
397     7612    0.983938
user_id     7645.000000
item_id    25559.000000
time           0.983999
Name: 338910, dtype: float64
     user_id  query_time
398     7645    0.983953
user_id     7667.000000
item_id    37541.000000
time           0.984008
Name: 251638, dtype: float64
     user_id  query_time
399     7667    0.983956
user_id     7678.000000
item_id    57491.000000
time           0.984004
Name: 340871, dtype: float64
  

...

user_id    27687.000000
item_id    70744.000000
time           0.984012
Name: 305274, dtype: float64
      user_id  query_time
1335    27687    0.983927
user_id    27753.000000
item_id    37885.000000
time           0.984005
Name: 405531, dtype: float64
      user_id  query_time
1341    27753    0.983954
user_id    27764.000000
item_id    31726.000000
time           0.984009
Name: 412662, dtype: float64
      user_id  query_time
1342    27764    0.983954
user_id    27775.000000
item_id    69353.000000
time           0.984009
Name: 341554, dtype: float64
      user_id  query_time
1343    27775    0.983929
user_id    27786.000000
item_id    30065.000000
time           0.983999
Name: 309995, dtype: float64
      user_id  query_time
1344    27786    0.983955
user_id    27819.000000
item_id    17616.000000
time           0.984006
Name: 294687, dtype: float64
      user_id  query_time
1346    27819     0.98395
user_id    27830.000000
item_id    86132.000000
time           0.984011
Name: 4600

user_id    30228.000
item_id    81907.000
time           0.984
Name: 335085, dtype: float64
      user_id  query_time
1451    30228    0.983936
user_id    30272.000000
item_id    42601.000000
time           0.983997
Name: 427820, dtype: float64
      user_id  query_time
1453    30272    0.983957
user_id    30492.000000
item_id    42045.000000
time           0.983942
Name: 383890, dtype: float64
      user_id  query_time
1458    30492    0.983941
user_id    30503.000000
item_id    22405.000000
time           0.983997
Name: 441343, dtype: float64
      user_id  query_time
1459    30503    0.983947
user_id    30514.000000
item_id    47673.000000
time           0.984002
Name: 406160, dtype: float64
      user_id  query_time
1460    30514    0.983951
user_id    30525.000
item_id    84438.000
time           0.984
Name: 349947, dtype: float64
      user_id  query_time
1461    30525    0.983874
user_id    30547.000000
item_id    30694.000000
time           0.984006
Name: 449305, dtype: float64

user_id    33110.000000
item_id    37596.000000
time           0.983984
Name: 259103, dtype: float64
      user_id  query_time
1557    33110      0.9839
user_id    33154.000000
item_id    33642.000000
time           0.984007
Name: 442020, dtype: float64
      user_id  query_time
1558    33154    0.983927
user_id    33165.000000
item_id    39466.000000
time           0.984012
Name: 310576, dtype: float64
      user_id  query_time
1559    33165    0.983947
user_id    33275.000000
item_id    25474.000000
time           0.984001
Name: 287931, dtype: float64
      user_id  query_time
1566    33275    0.983946
user_id    33297.000000
item_id    82543.000000
time           0.983985
Name: 298640, dtype: float64
      user_id  query_time
1567    33297    0.983943
user_id    33440.000000
item_id    67697.000000
time           0.983975
Name: 341456, dtype: float64
      user_id  query_time
1572    33440    0.983951
user_id    33473.000000
item_id    98837.000000
time           0.983964
Name: 2776

user_id     4500.000000
item_id    42083.000000
time           0.983946
Name: 10032, dtype: float64
      user_id  query_time
1883     4500    0.983845
user_id     5402.000000
item_id    25119.000000
time           0.983954
Name: 118955, dtype: float64
      user_id  query_time
1938     5402    0.983896
user_id     5600.000000
item_id    10026.000000
time           0.983942
Name: 22830, dtype: float64
      user_id  query_time
1948     5600    0.983942
user_id    5633.000000
item_id    7247.000000
time          0.983898
Name: 88430, dtype: float64
      user_id  query_time
1950     5633    0.983894
user_id     5842.000000
item_id    35165.000000
time           0.983942
Name: 46493, dtype: float64
      user_id  query_time
1963     5842    0.983942
user_id     7459.000000
item_id    79122.000000
time           0.983943
Name: 4984, dtype: float64
      user_id  query_time
2042     7459    0.983943
user_id     7767.000000
item_id    10338.000000
time           0.983943
Name: 198396, dtype

KeyboardInterrupt: 

In [35]:
for uid in test_qtime_0_df['user_id'].tolist():
    try:
        # 因为test_qtime_0_df的uid不一定都在train_click_0_df中
        if train_click_0_df.loc[train_click_0_df['user_id']==uid].sort_values('time')['time'].iloc[-1] >= test_qtime_0_df[test_qtime_0_df['user_id']==uid]['query_time'].iloc[-1]:
            print(train_click_0_df.loc[train_click_0_df['user_id']==uid].sort_values('time').iloc[-1])
            print(test_qtime_0_df[test_qtime_0_df['user_id']==uid])
    except:
        pass

In [36]:
for uid in test_qtime_1_df['user_id'].tolist():
    try:
        # 因为test_qtime_1_df的uid不一定都在train_click_1_df中
        if train_click_1_df.loc[train_click_1_df['user_id']==uid].sort_values('time')['time'].iloc[-1] >= test_qtime_1_df[test_qtime_1_df['user_id']==uid]['query_time'].iloc[-1]:
            print(train_click_1_df.loc[train_click_1_df['user_id']==uid].sort_values('time').iloc[-1])
            print(test_qtime_1_df[test_qtime_1_df['user_id']==uid])
    except:
        pass

In [33]:
# test数据中test_click_df的时间都在test_qtime_df之前
for uid in test_qtime_df['user_id'].tolist():
#     print(test_click_df.loc[test_click_df['user_id']==uid].sort_values('time')['time'].iloc[-1])
#     print(test_qtime_df[test_qtime_df['user_id']==uid]['query_time'].iloc[-1])
    if test_click_df.loc[test_click_df['user_id']==uid].sort_values('time')['time'].iloc[-1] >= test_qtime_df[test_qtime_df['user_id']==uid]['query_time'].iloc[-1]:
        print('----------')
        print(test_click_df.loc[test_click_df['user_id']==uid].sort_values('time').iloc[-1])
        print(test_qtime_df[test_qtime_df['user_id']==uid])
        

In [25]:
test_qtime_df[test_qtime_df['user_id']==1]

,user_id,query_time
1663,1,0.983942


## 大部分test_qtime_df.user_id都没给出user feature!!!

In [55]:
test_qtime_df.shape

(3389, 2)

In [60]:
test_qtime_df[~test_qtime_df.user_id.isin(train_user_df.user_id.tolist())].shape

(2596, 2)

In [61]:
test_qtime_df[test_qtime_df.user_id.isin(train_user_df.user_id.tolist())].shape

(793, 2)